In [1]:
import getpass
import os

# langsmith - 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
# openai - sk-proj-dZsrzaldTWL8F2aOFasJT3BlbkFJRZLVk9OWZh9WG66PArwp
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

In [4]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [6]:
# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("human", "{input}")]
)

In [7]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [8]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

'Task Decomposition is the process of breaking down a complex task into smaller, more manageable steps or subtasks. It includes methods like Chain of Thought (CoT) and Tree of Thoughts, which involve decomposing the problem into multiple thought steps and generating multiple thoughts per step. Task decomposition can be done by using simple prompts, task-specific instructions, or human inputs. This technique helps in enhancing model performance on complex tasks by making them easier to understand and execute.'

In [9]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(llm, retriever, q_prompt)

In [10]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


# The qa prompt needs to have the {context} is
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [11]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition"

msg1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([HumanMessage(content=question), AIMessage(content=msg1["answer"])])

second_question = "What are the common ways of doing it?"
msg2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
print(msg2["answer"])

Common ways of performing task decomposition include: (1) using Language Learning Models (LLM) with simple prompting, such as "Steps for XYZ.\n1." or "What are the subgoals for achieving XYZ?", (2) using task-specific instructions, for example "Write a story outline." for writing a novel, and (3) with human inputs. These methods help to break down complex tasks into smaller, manageable steps.


In [18]:
# You can then combine this with the RunnableWithmessagehistory that we saw from before to remember the inputs, history and blah blah
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_message_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [19]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task Decomposition is a process in which a complex task is broken down into smaller, simpler steps, making the task more manageable. Techniques like Chain of Thought (CoT) and Tree of Thoughts can be used for this process. CoT instructs the model to think step by step, transforming large tasks into multiple smaller tasks, while Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure.'

In [20]:
conversational_rag_chain.invoke(
    {"input": "What are the common ways of doing it"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition can be done in several ways: (1) by using large language models (LLMs) with simple prompting like "Steps for XYZ. 1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions such as "Write a story outline." for writing a novel, or (3) with human inputs. Methods like Chain of Thought (CoT) and Tree of Thoughts can also be used to decompose tasks into simpler steps.'